In [61]:
import nltk
from nltk.corpus import *
from nltk.stem.porter import *
import os
from Constants import *
import pickle
from nltk.tokenize import sent_tokenize, word_tokenize
from pathlib import Path

In [62]:
# stemming porter object
stemmer = PorterStemmer()
root = Path(".")

In [63]:
stop_words = set(stopwords.words('english'))

In [64]:
my_path = root / "Pickled_files" / "Inverted_index"
dbfile = open(my_path, 'rb')     
inverted_idx = pickle.load(dbfile)
dbfile.close()

my_path = root / "Pickled_files" / "Documents"
dbfile = open(my_path, 'rb')     
docs = pickle.load(dbfile)
dbfile.close()

my_path = root / "Pickled_files" / "Files"
dbfile = open(my_path, 'rb')     
files = pickle.load(dbfile)
dbfile.close()

In [65]:
def isnumber(token):  
    for char in token:
        if not (char >= '0' and char <= '9'):
            return False
    return True

In [66]:
def insert(word , root) : 
    cur = 0
    for char in word : 
        if root[cur].get(char) == None : 
            root[cur][char] = len(root)
            root.append(dict())
        cur = root[cur][char]
    root[cur]["end"] = True
    if root[cur].get("cnt") == None : 
        root[cur]["cnt"] = 0
    root[cur]["cnt"] += 1

In [67]:
def wildcard(word, root , is_suf) : 
    ans = list()
    cur = 0
    for char in word : 
        if root[cur].get(char) == None : 
            return ans
        cur = root[cur][char]
    vis = dict()
    def dfs(token , id) : 
        if vis.get(id) != None : 
            return
        vis[id] = 1
        if root[id].get("cnt") != None:
            if is_suf : 
                token = token[::-1]
            ans.append((token , root[id]["cnt"]))
        for labels in root[id].keys() :
            if len(labels) > 1 :
                continue
            new_str = token + labels
            dfs(new_str , root[id][labels])
    dfs(word , cur)
    ans = sorted(ans , key = lambda y : -y[1])
    ret_list = list()
    for i in range(min(3 , len(ans))) :
        ret_list.append(ans[i])
    return ret_list

In [68]:
Prefix = list()
Prefix.append(dict())
Suffix = list()
Suffix.append(dict())

In [69]:
for sentence in docs: 
    sentence = sentence[0]
    tokens = nltk.tokenize.word_tokenize(str(sentence))
    new_token = list()
    for i in tokens:
        new_token.append(i.lower())
    tokens = new_token
    for words in new_token: 
        if words in stop_words or len(words) <= 2 or isnumber(words): 
            continue
        insert(words , Prefix)
        insert(words[::-1] , Suffix)

In [70]:
my_path = root / "Pickled_files" / "Prefix_wildcard"
dbfile = open(my_path, 'wb')
pickle.dump(Prefix, dbfile) 
dbfile.close()

my_path = root / "Pickled_files" / "Suffix_wildcard"
dbfile = open(my_path, 'wb')
pickle.dump(Suffix, dbfile) 
dbfile.close()

In [71]:
my_path = root / "Pickled_files" / "Prefix_wildcard"
dbfile = open(my_path, 'rb')     
Prefix = pickle.load(dbfile)
dbfile.close()

my_path = root / "Pickled_files" / "Suffix_wildcard"
dbfile = open(my_path, 'rb')     
Suffix = pickle.load(dbfile)
dbfile.close()

In [72]:
def query(word , type) : 
    ans = set()
    if type : 
        word_list = wildcard(word , Suffix , True)
    else :
        word_list = wildcard(word , Prefix , False)
    for words in word_list :
        normalized_word = stemmer.stem(words[0])
        if inverted_idx.get(normalized_word) != None : 
            for i in range(min(len(inverted_idx[normalized_word]), 2)):
                ans.add((docs[inverted_idx[normalized_word][i][0]][0], files[inverted_idx[normalized_word][i][1]]))
    return ans

In [73]:
results = query("ill" , False)
for i in results : 
    print(i[1])
    print(i[0])

.\Docs\Property\Property-Owner-Policy-Wording.docx
documents documents belonging to the insured or held by the insured in trust , whilst at premises not being the insured ’ s premises , or in transit by road , rail or inland waterway.specified illnesses contingencies any occurrence of a specified illness at the premises , except where the premises is a private dwelling any discovery of an organism at the premises likely to result in the occurrence of a specified illness , except where the premises is a private dwelling any occurrence of legionellosis at the premises d the discovery of vermin or pests at the premises e any accident causing defects in the drains or other sanitary arrangements at the premises which causes restrictions on the use of the premises on the order or advice of the competent local authority.special provisions ‘ specified illness ’ shall mean illness sustained by any person resulting from food or drink poisoning , or acute encephalitis , acute poliomyelitis , anth